In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Process

## Price

Source: https://www.smard.de/home/downloadcenter/download-marktdaten/

In [2]:
file_path = '0_raw/Gro_handelspreise_201507010000_201807010000_Stunde.csv'
raw_df = pd.read_csv(file_path, sep=';', header=0, converters={'DE/AT/LU [€/MWh] Originalauflösungen': lambda x: float(x.replace(",", "."))})
raw_values = raw_df['DE/AT/LU [€/MWh] Originalauflösungen']
timestep_raw_values = pd.Series([x/100 for x in raw_values for _ in range(2)]).round(4)

start = datetime(2015, 7, 1, 0, 30)
end = datetime(2018, 7, 1, 0, 0)
dates = pd.date_range(start, end, freq='30min')

price = pd.DataFrame({'Date': dates, 'Price': timestep_raw_values}, columns=['Date', 'Price'])

start = datetime(2016, 2, 29, 0, 30) 
end = datetime(2016, 3, 1, 0, 0)
remove = pd.date_range(start, end, freq='30min')
price_wo_leap = price[~price['Date'].isin(remove)]

price_wo_leap.to_csv('1_processed/price.csv', index=False)
price_wo_leap

,Date,Price
0,2015-07-01 00:30:00,0.3062
1,2015-07-01 01:00:00,0.3062
2,2015-07-01 01:30:00,0.2998
3,2015-07-01 02:00:00,0.2998
4,2015-07-01 02:30:00,0.2805
...,...,...
52603,2018-06-30 22:00:00,0.4751
52604,2018-06-30 22:30:00,0.4780
52605,2018-06-30 23:00:00,0.4780
52606,2018-06-30 23:30:00,0.4396


## Load & PV

In [6]:
file_paths = [
    '0_raw/2010-2011 Solar home electricity data.csv',
    '0_raw/2011-2012 Solar home electricity data.csv',
    '0_raw/2012-2013 Solar home electricity data.csv']

dataframe = pd.DataFrame()

for file_path in file_paths:
    df = pd.read_csv(file_path, header=1)

    if '2010-2011' in file_path:
        df.date = pd.to_datetime(df.date, format='%d-%b-%y')
    else:
        df.date = pd.to_datetime(df.date, format='%d/%m/%Y')
    
    if 'Row Quality' in df.columns:
        df.drop('Row Quality', axis=1, inplace=True)
    df.drop('Generator Capacity', axis=1, inplace=True)
    df.drop('Postcode', axis=1, inplace=True)

    # Append the modified dataframe to the list
    dataframe = pd.concat([dataframe, df], ignore_index=True)

energy = {}
start = datetime(2010, 7, 1, 0, 30)
end = datetime(2013, 7, 1, 0, 0)
dates = pd.date_range(start, end, freq='30min')
energy['Date'] = dates


for customer in dataframe['Customer'].unique():
    df = dataframe[dataframe['Customer'] == customer]
    melted = df.melt(id_vars=['Customer','Consumption Category','date'],  var_name='time', value_name='value')
    melted['time'] = melted['time'].replace('0:00', '24:00')
    melted['datetime'] = pd.to_datetime(melted["date"] + pd.to_timedelta(melted['time']+':00'))

    df_load = melted[melted["Consumption Category"].isin(["GC", "CL"])].copy()
    df_load.drop("Customer",axis=1, inplace=True)
    df_load_grouped = df_load.groupby(["datetime"])["value"].sum()

    df_pv = melted[melted["Consumption Category"].isin(["GG"])].copy()
    df_pv.drop("Customer",axis=1, inplace=True)
    df_pv_grouped = df_pv.groupby(["datetime"])["value"].sum()
    
    energy[f'load_{customer}'] = pd.Series(df_load_grouped.values).round(3)
    energy[f'pv_{customer}'] = pd.Series(df_pv_grouped.values).round(3)
    energy[f'prosumption_{customer}'] = pd.Series(df_load_grouped.values - df_pv_grouped.values).round(3)
    
energy = pd.DataFrame(energy)
start = datetime(2012, 2, 29, 0, 30) 
end = datetime(2012, 3, 1, 0, 0)
remove = pd.date_range(start, end, freq='30min')
energy_wo_leap = energy[~energy['Date'].isin(remove)]
energy_wo_leap.to_csv('1_processed/energy.csv', index=False)
energy_wo_leap

,Date,load_1,pv_1,prosumption_1,load_2,pv_2,prosumption_2,load_3,pv_3,prosumption_3,...,prosumption_297,load_298,pv_298,prosumption_298,load_299,pv_299,prosumption_299,load_300,pv_300,prosumption_300
0,2010-07-01 00:30:00,1.553,0.0,1.553,0.085,0.0,0.085,2.698,0.0,2.698,...,0.205,0.096,0.0,0.096,1.464,0.0,1.464,2.766,0.0,2.766
1,2010-07-01 01:00:00,1.715,0.0,1.715,0.142,0.0,0.142,2.684,0.0,2.684,...,0.106,0.110,0.0,0.110,1.413,0.0,1.413,3.316,0.0,3.316
2,2010-07-01 01:30:00,1.339,0.0,1.339,0.082,0.0,0.082,1.675,0.0,1.675,...,0.169,0.080,0.0,0.080,1.428,0.0,1.428,2.088,0.0,2.088
3,2010-07-01 02:00:00,0.865,0.0,0.865,0.119,0.0,0.119,0.683,0.0,0.683,...,0.270,0.105,0.0,0.105,1.445,0.0,1.445,0.066,0.0,0.066
4,2010-07-01 02:30:00,0.380,0.0,0.380,0.105,0.0,0.105,0.694,0.0,0.694,...,0.255,0.089,0.0,0.089,1.407,0.0,1.407,0.089,0.0,0.089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52603,2013-06-30 22:00:00,0.274,0.0,0.274,NaN,NaN,NaN,0.056,0.0,0.056,...,0.167,0.267,0.0,0.267,0.618,0.0,0.618,0.581,0.0,0.581
52604,2013-06-30 22:30:00,0.509,0.0,0.509,NaN,NaN,NaN,0.056,0.0,0.056,...,0.954,0.273,0.0,0.273,0.568,0.0,0.568,0.458,0.0,0.458
52605,2013-06-30 23:00:00,0.166,0.0,0.166,NaN,NaN,NaN,0.065,0.0,0.065,...,2.594,0.247,0.0,0.247,0.832,0.0,0.832,0.472,0.0,0.472
52606,2013-06-30 23:30:00,0.861,0.0,0.861,NaN,NaN,NaN,0.063,0.0,0.063,...,1.266,0.149,0.0,0.149,0.760,0.0,0.760,0.732,0.0,0.732
